In [1]:
#@title Load requirements
!git clone https://github.com/NetoIE/ML_PMU_Library --quiet
%cd /content/ML_PMU_Library

import matplotlib.pyplot as plt
import ipywidgets as widgets
import functools
import itertools
import pandas as pd
import numpy as np
import anomalyDetClass
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix


/content/ML_PMU_Library


# Monte Carlo model

In [2]:
#----------Montecarlo Model---------
nA = 10000 #anomalies per class
beta = (0.005, 0.01) #magnitude range
T = (30,120) #temporality range
types = [0, 1, 2] #pulse, ramp, step
rs = np.random.RandomState(seed=(1,2,3)) #random generator
anomalies = anomalyDetClass.generate_anomalies(nA,beta,T,[True,True,True],rs)

#----------Create signal with anomalies and noise---------
wS = 60 #windows size
SNR = 60 #Signal to noise ration in dB
rs = np.random.RandomState(seed=(1,2,3)) #random generator
signal = anomalyDetClass.generate_signal(anomalies, nA, types, wS, SNR, rs)

# Training stage

In [3]:
#-------Prepare data for detector and classifier models
rs = np.random.RandomState(seed=(1,2,3))  #random generator
#noisy windows of same length than total anomalies for test purposes
signal_inliers = anomalyDetClass.awgn(np.ones(anomalies.shape[0] * wS),SNR,rs)

_tags_array = np.repeat([1,0], 2 * len(types) * nA) #detector flags 
tags_array = np.repeat(types,nA) # classifier tags (pulse/ramp/step)

shuffled_indices = np.arange(len(types)*nA).reshape(-1,nA)
for i in range(len(types)):
  rs.shuffle(shuffled_indices[i,:])  

#split train/test data in a 80/20 rate
idx = int(0.8 * nA)
train_idx = shuffled_indices[:,:idx].flatten()
test_idx = shuffled_indices[:,idx:].flatten()

#Outliers complete windows indices
a_len = np.array([anomaly.shape[0] for anomaly in anomalies]) #anomaly length
nd = np.cumsum(a_len + 2 * wS) #anomaly end index position at signal
st = nd - a_len #anomaly start index position at signal

#Noisy windows indices
sst = np.array([wS * i for i in range(anomalies.shape[0])]) #noisy window start index position at signal_inliers
nnd = sst + wS #noisy window end index position at signal_inliers


#--------- detector parameters' calibration -----

m = 3 #embedded dimension for phase space reconstruction
tau = 1 #time delay for phase space reconstruction
Nphsp = wS - (m - 1) * tau  #number of vectors from phase space
k = int(0.3*Nphsp) #k value for k-distance
E_th = Nphsp ** 2 * (Nphsp - 1) #complete graph laplacian energy (detector threshold)
eps = 0.8 * 0.005 #detector k-dist recurrence graph threshold

#------ SVM classifier training stage-----

rs = np.random.RandomState(seed=(1,2,3)) 
def get_features(x):  
  eps = 3 * np.std(x) #classifier recurrence graph threshold
  r_plot = anomalyDetClass.recurrencePlot(x,m,tau,eps) # Euclidean recurrence plot
  features = anomalyDetClass.featureExtractionRQA(r_plot,1,2)
  return features

X_train = np.array([get_features(signal[i:j]) for i,j in zip(st[train_idx],nd[train_idx])])
y_train = tags_array[train_idx]

# features subspaces for each pair of classes
# RR, dmean, dMax, vmean, vMax -- 0,1,2,3,4
features_subspace = np.array([
[0,2,3], #pulse-ramp
[0,1,4], #pulse-step
[0,1,3]  #step-ramp                            
])

#Prefer dual=False when samples > features.
total_models = int(len(types) * (len(types) - 1) / 2)
DAGSVM = [LinearSVC(dual=False,random_state=rs) for i in range(total_models)]

#fit SVM models to samples
i = 0
for c in itertools.combinations(types, 2):  
  id = np.r_[idx * c[0]:idx * (c[0]+1), idx * c[1]:idx * (c[1]+1)]
  DAGSVM[i] = DAGSVM[i].fit(X_train[id,:][:,features_subspace[i]], y_train[id])    
  i += 1

# Detector test

In [4]:
rs = np.random.RandomState(seed=(1,2,3)) 
titles = ['Pulse', 'Ramp', 'Step']

def split_windows(st,nd = wS):  
  #split anomaly in at least one window
  ws = 1 #window step
  wn = max(int((nd-st - wS)/ws)+1,1) #Number of windows   
  windows = (st + ws * i for i in range(wn)) #retain only start index
  return windows

def energy(adj_mat):
  deg = np.sum(adj_mat,0)    
  normalized_energy = np.sum(deg ** 2 + deg) / ( Nphsp ** 2 * (Nphsp-1))  
  return normalized_energy

def detect(X,st,nd): #recurrence rate  
  windows = split_windows(st,nd)          
  det = np.array([ energy(anomalyDetClass.k_dist_RPlot(
      X[w_st:w_st+wS],m,tau,k,eps)) < 1.0 for w_st in windows])    
  # return np.round(np.mean(det)) #full anomaly captured > 50 %    
  return np.any(det) #anomaly captured  

#--------------------Detection evaluation score--------------------------

y_pred = np.array([detect(signal,i,j) for i,j in zip(st[test_idx],nd[test_idx])] + [detect(signal_inliers,i,j) for i,j in zip(sst[test_idx],nnd[test_idx])])

def evaluation_score(y_test,y_pred,title):
  conf_matrix =  confusion_matrix(y_test, y_pred)
  TP = np.diag(conf_matrix)
  FP = conf_matrix.sum(axis=0) - TP  
  FN = conf_matrix.sum(axis=1) - TP
  TN = conf_matrix.sum() - (FP + FN + TP)

  TPR = TP / (TP + FN)
  TNR = TN / (TN + FP)
  FPR = 1 - TNR
  PREC = TP / (TP + FP)
  F1 = 2 * TP / (2 * TP + FN + FP)
  JACCARD = TP / (TP + FP + FN)
  ACC = (TP + TN) / (TP+FP+FN+TN)

  df = pd.DataFrame(np.array([TPR,TNR,FPR,PREC,F1,JACCARD,ACC]).T,
                    columns=['TPR','TNR','FPR','PREC','F1','JACCARD','ACC'])
  df = df.loc[df.index==0].style.set_caption(title).hide_index()  
  display(df)

test_len = nA - idx
for i,_ in enumerate(types):
  i0 = i*test_len  
  i1 = (i+1)*test_len    
  det_pred = np.append(y_pred[i0:i1],y_pred[-len(types) * test_len:][i0:i1])
  evaluation_score(np.repeat([1,0],test_len),det_pred, titles[i])  


TPR,TNR,FPR,PREC,F1,JACCARD,ACC
0.988500,1.000000,0.000000,1.000000,0.994217,0.988500,0.994250


TPR,TNR,FPR,PREC,F1,JACCARD,ACC
0.983500,0.973000,0.027000,0.973281,0.978364,0.957644,0.978250


TPR,TNR,FPR,PREC,F1,JACCARD,ACC
0.983000,1.000000,0.000000,1.000000,0.991427,0.983000,0.991500


# Classifier test

In [5]:
#--------------------Classification evaluation score--------------------------
def evaluation_score(y_test,y_pred,title):
  titles = ['Pulse', 'Ramp', 'Step']
  conf_matrix =  confusion_matrix(y_test, y_pred)
  TP = np.diag(conf_matrix)
  FP = conf_matrix.sum(axis=0) - TP  
  FN = conf_matrix.sum(axis=1) - TP
  TN = conf_matrix.sum() - (FP + FN + TP)

  TPR = TP / (TP + FN)
  TNR = TN / (TN + FP)
  FPR = 1 - TNR
  PREC = TP / (TP + FP)
  F1 = 2 * TP / (2 * TP + FN + FP)
  JACCARD = TP / (TP + FP + FN)
  ACC = (TP + TN) / (TP+FP+FN+TN)

  df = pd.DataFrame(np.array([TPR,TNR,FPR,PREC,F1,JACCARD,ACC]).T,
                    columns=['TPR','TNR','FPR','PREC','F1','JACCARD','ACC'],
                    index=titles)
  df = df.style.set_caption(title)
  display(df)

X_test = np.array([get_features(signal[i:j]) for i,j in zip(st[test_idx],nd[test_idx])])
y_test = tags_array[test_idx]

y_pred = DAGSVM[0].predict(X_test[:,features_subspace[0]]) 
c1 = y_pred == 0
c2 = np.invert(c1)
y_pred[c1] = DAGSVM[1].predict(X_test[c1,:][:,features_subspace[1]])
y_pred[c2] = DAGSVM[2].predict(X_test[c2,:][:,features_subspace[2]])

evaluation_score(y_test,y_pred,'DAGSVM')

,TPR,TNR,FPR,PREC,F1,JACCARD,ACC
Pulse,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
Ramp,0.990000,0.996000,0.004000,0.991984,0.990991,0.982143,0.994000
Step,0.992000,0.995000,0.005000,0.990020,0.991009,0.982178,0.994000
